In [1]:
import gc
import io
import os
import time
import sys
sys.path.append("../")
import numpy as np
import tensorflow as tf
import tensorflow_gan as tfgan
import logging
# Keep the import below for registering all model definitions
from models import ddpm#, ncsnpp
import losses
import sampling
from models import utils as mutils
from models.ema import ExponentialMovingAverage
import datasets
import evaluation
import likelihood
import sde_lib
from absl import flags
import torch
from torch.utils import tensorboard
from torchvision.utils import make_grid, save_image
from utils import save_checkpoint, restore_checkpoint
from configs.vp import cifar10_ddpm_less as configs
from sde_lib import VESDE, VPSDE, subVPSDE
from torch.profiler import profile, record_function, ProfilerActivity

/home/yifulu/.conda/envs/dpm/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
config=configs.get_config()
config.eval.batch_size = 16
sde = VPSDE(beta_min=config.model.beta_min, beta_max=config.model.beta_max, N=config.model.num_scales)
sampling_eps = 1e-3
# Build data pipeline
train_ds, eval_ds, _ = datasets.get_dataset(config,
                                            uniform_dequantization=config.data.uniform_dequantization,
                                            evaluation=True)

# Create data normalizer and its inverse
scaler = datasets.get_data_scaler(config)
inverse_scaler = datasets.get_data_inverse_scaler(config)

No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)
2023-03-27 12:18:23.842330: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /sw/pkgs/arc/cudnn/11.3-v8.2.1/lib64:/sw/pkgs/arc/cuda/11.3.0/lib64:/opt/slurm/lib64::
2023-03-27 12:18:23.842363: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)


In [3]:
score_model = mutils.create_model(config)
optimizer = losses.get_optimizer(config, score_model.parameters())
ema = ExponentialMovingAverage(score_model.parameters(), decay=config.model.ema_rate)
state = dict(optimizer=optimizer, model=score_model, ema=ema, step=0)


sde = VPSDE(beta_min=config.model.beta_min, beta_max=config.model.beta_max, N=config.model.num_scales)
sampling_eps = 1e-3


In [4]:
sampling_shape = (config.eval.batch_size,
                      config.data.num_channels,
                      config.data.image_size, config.data.image_size)
eval_iter = iter(eval_ds)

In [5]:
batch = torch.from_numpy(next(eval_iter)['image']._numpy()).to(config.device).float()
batch = batch.permute(0, 3, 1, 2)
batch = scaler(batch)
lst_steps=torch.tensor([0.050950001925230026])
t = lst_steps.to(batch.device).repeat(batch.shape[0])
z = torch.randn_like(batch)
mean, std = sde.marginal_prob(batch, t)
perturbed_data = mean + std[:, None, None, None] * z

In [6]:
with profile(activities=[ProfilerActivity.CPU], record_shapes=True,with_flops=True) as prof:
    with record_function("model_inference"):
        score_model(perturbed_data,t)

STAGE:2023-03-27 12:18:25 2564023:2564023 ActivityProfilerController.cpp:294] Completed Stage: Warm Up
STAGE:2023-03-27 12:18:25 2564023:2564023 ActivityProfilerController.cpp:300] Completed Stage: Collection


In [7]:
print(prof.key_averages().table(sort_by="cpu_time_total", row_limit=10))

----------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                        Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg    # of Calls  Total MFLOPs  
----------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
             model_inference         2.47%       6.686ms        99.98%     270.216ms     270.216ms             1            --  
        DataParallel.forward         5.18%      13.987ms        97.51%     263.525ms     263.525ms             1            --  
                aten::conv2d         0.74%       2.011ms        33.93%      91.702ms       1.764ms            52     98354.332  
           aten::convolution         0.33%     888.000us        33.81%      91.368ms       1.757ms            52            --  
          aten::_convolution         0.25%     676.000us        33.48%      90.480ms       1.740m